In [3]:
"""
Fuzzy Logic Implementation for Dengue Hemorrhagic Fever (DBD) Risk Diagnosis
Implementasi Fuzzy Logic untuk Diagnosa Risiko Demam Berdarah (DBD)

Author: [Rifki Setiawan - G.211.22.0128]
Date: December 2025
Course: UAS Fuzzy Logic - Gasal 2024/2025
"""

import numpy as np


def membership_suhu_normal(x):
    """Membership function untuk suhu normal (36-37.5°C)"""
    if x <= 37:
        return 1.0
    elif 37 < x < 37.5:
        return (37.5 - x) / 0.5
    else:
        return 0.0


def membership_suhu_demam(x):
    """Membership function untuk suhu demam (37-39°C)"""
    if x <= 37:
        return 0.0
    elif 37 < x <= 38:
        return (x - 37) / 1.0
    elif 38 < x <= 39:
        return (39 - x) / 1.0
    else:
        return 0.0


def membership_suhu_tinggi(x):
    """Membership function untuk suhu tinggi (>38.5°C)"""
    if x <= 38.5:
        return 0.0
    elif 38.5 < x <= 39.5:
        return (x - 38.5) / 1.0
    else:
        return 1.0


def membership_trombosit_rendah(x):
    """Membership function untuk trombosit rendah (<150)"""
    if x <= 100:
        return 1.0
    elif 100 < x < 150:
        return (150 - x) / 50
    else:
        return 0.0


def membership_trombosit_normal(x):
    """Membership function untuk trombosit normal (>120)"""
    if x <= 120:
        return 0.0
    elif 120 < x <= 150:
        return (x - 120) / 30
    else:
        return 1.0


def membership_leukosit_rendah(x):
    """Membership function untuk leukosit rendah (<4.5)"""
    if x <= 3.5:
        return 1.0
    elif 3.5 < x < 4.5:
        return (4.5 - x) / 1.0
    else:
        return 0.0


def membership_leukosit_normal(x):
    """Membership function untuk leukosit normal (>4)"""
    if x <= 4:
        return 0.0
    elif 4 < x <= 5:
        return (x - 4) / 1.0
    else:
        return 1.0


def fuzzy_dbd_diagnosis(suhu_val, trombosit_val, leukosit_val, verbose=True):
    """
    Fungsi utama untuk diagnosa DBD menggunakan Fuzzy Logic

    Parameters:
    -----------
    suhu_val : float
        Suhu tubuh pasien (°C)
    trombosit_val : float
        Jumlah trombosit (10⁹/L)
    leukosit_val : float
        Jumlah leukosit (10⁹/L)
    verbose : bool
        Tampilkan hasil detail (default: True)

    Returns:
    --------
    dict : Dictionary berisi hasil fuzzifikasi, rules, agregasi, dan nilai risiko
    """

    # Fuzzifikasi input
    suhu_normal = membership_suhu_normal(suhu_val)
    suhu_demam = membership_suhu_demam(suhu_val)
    suhu_tinggi = membership_suhu_tinggi(suhu_val)

    trombosit_rendah = membership_trombosit_rendah(trombosit_val)
    trombosit_normal = membership_trombosit_normal(trombosit_val)

    leukosit_rendah = membership_leukosit_rendah(leukosit_val)
    leukosit_normal = membership_leukosit_normal(leukosit_val)

    # Evaluasi Rules (menggunakan min untuk AND)
    # Rule 1: Jika Suhu tinggi AND Trombosit rendah AND Leukosit rendah maka Risiko = Tinggi
    rule1 = min(suhu_tinggi, trombosit_rendah, leukosit_rendah)

    # Rule 2: Jika Suhu demam AND Trombosit rendah maka Risiko = Tinggi
    rule2 = min(suhu_demam, trombosit_rendah)

    # Rule 3: Jika Suhu demam AND Trombosit normal maka Risiko = Sedang
    rule3 = min(suhu_demam, trombosit_normal)

    # Rule 4: Jika Suhu normal AND Trombosit normal AND Leukosit normal maka Risiko = Rendah
    rule4 = min(suhu_normal, trombosit_normal, leukosit_normal)

    # Rule 5: Jika Suhu tinggi AND Leukosit normal maka Risiko = Sedang
    rule5 = min(suhu_tinggi, leukosit_normal)

    # Agregasi (menggunakan max untuk OR)
    risiko_tinggi = max(rule1, rule2)
    risiko_sedang = max(rule3, rule5)
    risiko_rendah = rule4

    # Defuzzifikasi menggunakan metode Weighted Average
    if risiko_tinggi + risiko_sedang + risiko_rendah == 0:
        nilai_risiko = 0
    else:
        nilai_risiko = (risiko_tinggi * 85 + risiko_sedang * 50 + risiko_rendah * 15) / \
                       (risiko_tinggi + risiko_sedang + risiko_rendah)

    hasil = {
        'fuzzifikasi': {
            'suhu': {'normal': suhu_normal, 'demam': suhu_demam, 'tinggi': suhu_tinggi},
            'trombosit': {'rendah': trombosit_rendah, 'normal': trombosit_normal},
            'leukosit': {'rendah': leukosit_rendah, 'normal': leukosit_normal}
        },
        'rules': {
            'rule1': rule1,
            'rule2': rule2,
            'rule3': rule3,
            'rule4': rule4,
            'rule5': rule5
        },
        'agregasi': {
            'risiko_tinggi': risiko_tinggi,
            'risiko_sedang': risiko_sedang,
            'risiko_rendah': risiko_rendah
        },
        'nilai_risiko': nilai_risiko
    }

    if verbose:
        print_hasil_fuzzy(suhu_val, trombosit_val, leukosit_val, hasil)

    return hasil


def print_hasil_fuzzy(suhu, trombosit, leukosit, hasil):
    """Print hasil diagnosa fuzzy dengan format yang rapi"""

    print("="*70)
    print("FUZZY LOGIC - DIAGNOSA RISIKO DEMAM BERDARAH (DBD)")
    print("="*70)
    print("\n1. DATA PASIEN")
    print("-"*70)
    print(f"   Suhu Tubuh    : {suhu}°C")
    print(f"   Trombosit     : {trombosit} x10⁹/L")
    print(f"   Leukosit      : {leukosit} x10⁹/L")

    print("\n2. FUZZIFIKASI")
    print("-"*70)
    print("   Suhu:")
    print(f"     - Normal : {hasil['fuzzifikasi']['suhu']['normal']:.4f}")
    print(f"     - Demam  : {hasil['fuzzifikasi']['suhu']['demam']:.4f}")
    print(f"     - Tinggi : {hasil['fuzzifikasi']['suhu']['tinggi']:.4f}")
    print("\n   Trombosit:")
    print(f"     - Rendah : {hasil['fuzzifikasi']['trombosit']['rendah']:.4f}")
    print(f"     - Normal : {hasil['fuzzifikasi']['trombosit']['normal']:.4f}")
    print("\n   Leukosit:")
    print(f"     - Rendah : {hasil['fuzzifikasi']['leukosit']['rendah']:.4f}")
    print(f"     - Normal : {hasil['fuzzifikasi']['leukosit']['normal']:.4f}")

    print("\n3. EVALUASI RULES")
    print("-"*70)
    print(f"   Rule 1 (Tinggi & Rendah & Rendah → Tinggi) : {hasil['rules']['rule1']:.4f}")
    print(f"   Rule 2 (Demam & Rendah → Tinggi)           : {hasil['rules']['rule2']:.4f}")
    print(f"   Rule 3 (Demam & Normal → Sedang)           : {hasil['rules']['rule3']:.4f}")
    print(f"   Rule 4 (Normal & Normal & Normal → Rendah) : {hasil['rules']['rule4']:.4f}")
    print(f"   Rule 5 (Tinggi & Normal → Sedang)          : {hasil['rules']['rule5']:.4f}")

    print("\n4. AGREGASI")
    print("-"*70)
    print(f"   Risiko Tinggi : {hasil['agregasi']['risiko_tinggi']:.4f}")
    print(f"   Risiko Sedang : {hasil['agregasi']['risiko_sedang']:.4f}")
    print(f"   Risiko Rendah : {hasil['agregasi']['risiko_rendah']:.4f}")

    print("\n5. DEFUZZIFIKASI (Weighted Average)")
    print("-"*70)
    print(f"   Nilai Risiko  : {hasil['nilai_risiko']:.2f}")

    if hasil['nilai_risiko'] >= 70:
        kategori = "TINGGI"
        interpretasi = "Pasien memiliki RISIKO TINGGI DBD.\n   Rekomendasi: Segera lakukan penanganan medis dan rawat inap."
    elif hasil['nilai_risiko'] >= 40:
        kategori = "SEDANG"
        interpretasi = "Pasien memiliki RISIKO SEDANG DBD.\n   Rekomendasi: Monitoring ketat dan observasi berkala diperlukan."
    else:
        kategori = "RENDAH"
        interpretasi = "Pasien memiliki RISIKO RENDAH DBD.\n   Rekomendasi: Tetap jaga kesehatan dan istirahat cukup."

    print(f"   Kategori      : {kategori}")
    print(f"\n6. INTERPRETASI")
    print("-"*70)
    print(f"   {interpretasi}")
    print("="*70)


if __name__ == "__main__":
    # Test dengan data dari soal UAS
    print("\nContoh Kasus Pasien dari Soal UAS:\n")

    suhu_pasien = 38.5
    trombosit_pasien = 120
    leukosit_pasien = 3.8

    hasil = fuzzy_dbd_diagnosis(suhu_pasien, trombosit_pasien, leukosit_pasien)

    # Contoh kasus tambahan
    print("\n\nContoh Kasus Tambahan:\n")

    # Kasus 2: Risiko Rendah
    print("\nKasus 2: Pasien Sehat")
    fuzzy_dbd_diagnosis(36.8, 200, 7.0)

    # Kasus 3: Risiko Sedang
    print("\n\nKasus 3: Risiko Sedang")
    fuzzy_dbd_diagnosis(38.0, 170, 5.5)



Contoh Kasus Pasien dari Soal UAS:

FUZZY LOGIC - DIAGNOSA RISIKO DEMAM BERDARAH (DBD)

1. DATA PASIEN
----------------------------------------------------------------------
   Suhu Tubuh    : 38.5°C
   Trombosit     : 120 x10⁹/L
   Leukosit      : 3.8 x10⁹/L

2. FUZZIFIKASI
----------------------------------------------------------------------
   Suhu:
     - Normal : 0.0000
     - Demam  : 0.5000
     - Tinggi : 0.0000

   Trombosit:
     - Rendah : 0.6000
     - Normal : 0.0000

   Leukosit:
     - Rendah : 0.7000
     - Normal : 0.0000

3. EVALUASI RULES
----------------------------------------------------------------------
   Rule 1 (Tinggi & Rendah & Rendah → Tinggi) : 0.0000
   Rule 2 (Demam & Rendah → Tinggi)           : 0.5000
   Rule 3 (Demam & Normal → Sedang)           : 0.0000
   Rule 4 (Normal & Normal & Normal → Rendah) : 0.0000
   Rule 5 (Tinggi & Normal → Sedang)          : 0.0000

4. AGREGASI
----------------------------------------------------------------------
   R